In [ ]:
import requests
from datetime import datetime, timedelta
import xml.etree.ElementTree as ET
from statistics import mean
import time

def get_exchange_rates_for_period(days=90):
    base_url = "http://www.cbr.ru/scripts/XML_daily_eng.asp"
    all_rates = []
    processed_dates = set()  # Для обработанных дат

    print(f"Сбор данных за последние {days} дней")

    current_day = 0
    successful_days = 0

    while current_day < days:
        current_date = datetime.now() - timedelta(days=current_day)
        date_str = current_date.strftime("%d/%m/%Y")
        date_key = current_date.strftime("%Y-%m-%d")

        # Предварительный пропуск проверенных дат
        if date_key in processed_dates:
            print(f"# {date_str} уже обработана, предварительный пропуск")
            current_day += 1
            continue

        try:
            # Запрос к API + парсинг XML с извлечением даты
            response = requests.get(f"{base_url}?date_req={date_str}", timeout=10)
            response.raise_for_status()
            root = ET.fromstring(response.content)
            date_attribute = root.attrib.get('Date', date_str)

            # Проверка уникальности даты
            xml_date_key = datetime.strptime(date_attribute, "%d.%m.%Y").strftime("%Y-%m-%d")
            if xml_date_key in processed_dates:
                print(f"# {date_str} ({date_attribute}) уже обработана, пропуск")
                current_day += 1
                continue

            valutes = root.findall('Valute')
            day_rates = []
            for valute in valutes:
                currency_data = {
                    'date': date_attribute,
                    'date_original': current_date.strftime("%Y-%m-%d"),
                    'date_key': xml_date_key,
                    'currency_id': valute.attrib['ID'],
                    'num_code': valute.find('NumCode').text,
                    'char_code': valute.find('CharCode').text,
                    'nominal': int(valute.find('Nominal').text),
                    'name': valute.find('Name').text,
                    'value': float(valute.find('Value').text.replace(',', '.')),
                    'vunit_rate': float(valute.find('VunitRate').text.replace(',', '.'))
                }
                day_rates.append(currency_data)

            all_rates.extend(day_rates)
            processed_dates.add(xml_date_key)
            successful_days += 1
            print(f"+ {date_str} ({date_attribute}): получено {len(day_rates)} валют (день {successful_days}/{days})")

        except requests.exceptions.RequestException as e:
            print(f"Ошибка для даты {date_str}: {e}")
        except ET.ParseError as e:
            print(f"Ошибка парсинга XML для даты {date_str}: {e}")
        except AttributeError as e:
            print(f"Ошибка структуры XML для даты {date_str}: {e}")
        except ValueError as e:
            print(f"Ошибка формата даты для {date_str}: {e}")

        current_day += 1
        time.sleep(0.1)

    return all_rates

def analyze_currency_data(rates_data):
    if not rates_data:
        print("Нет данных")
        return None

    # Для вычислений использовал нормализованные значения из поля vunit_rate
    # 1. Максимальный курс
    max_rate = max(rates_data, key=lambda x: x['vunit_rate'])
    # 2. Минимальный курс
    min_rate = min(rates_data, key=lambda x: x['vunit_rate'])
    # 3. Среднее значение курса по всем валютам за весь период
    all_values_per_unit = [rate['vunit_rate'] for rate in rates_data]
    average_rate = mean(all_values_per_unit)

    # Получаем уникальные даты из обработанных данных
    unique_dates = sorted(set(rate['date_key'] for rate in rates_data))

    return {
        'max_currency': {
            'value': max_rate['value'],
            'vunit_rate': max_rate['vunit_rate'],
            'name': max_rate['name'],
            'date': max_rate['date'],
            'char_code': max_rate['char_code'],
            'nominal': max_rate['nominal']
        },
        'min_currency': {
            'value': min_rate['value'],
            'vunit_rate': min_rate['vunit_rate'],
            'name': min_rate['name'],
            'date': min_rate['date'],
            'char_code': min_rate['char_code'],
            'nominal': min_rate['nominal']
        },
        'average_rate': average_rate,
        'total_records': len(rates_data),
        'unique_currencies': len(set(rate['char_code'] for rate in rates_data)),
        'unique_dates': len(unique_dates),
        'date_range': {
            'start': min(unique_dates) if unique_dates else None,
            'end': max(unique_dates) if unique_dates else None
        }
    }

def print_detailed_results(results):
    print("\n" + "="*80)
    print("РЕЗУЛЬТАТЫ АНАЛИЗА КУРСОВ ВАЛЮТ ЦБ РФ")
    print("="*80)

    print(f"\n1. МАКСИМАЛЬНЫЙ КУРС ВАЛЮТЫ:")
    print(f"  Валюта: {results['max_currency']['name']} ({results['max_currency']['char_code']})")
    print(f"  Курс: {results['max_currency']['nominal']} {results['max_currency']['char_code']} = {results['max_currency']['value']:.6f} RUB")
    print(f"  За 1 единицу: {results['max_currency']['vunit_rate']:.6f} RUB")
    print(f"  Дата: {results['max_currency']['date']}")

    print(f"\n2. МИНИМАЛЬНЫЙ КУРС ВАЛЮТЫ:")
    print(f"  Валюта: {results['min_currency']['name']} ({results['min_currency']['char_code']})")
    print(f"  Курс: {results['min_currency']['nominal']} {results['min_currency']['char_code']} = {results['min_currency']['value']:.6f} RUB")
    print(f"  За 1 единицу: {results['min_currency']['vunit_rate']:.6f} RUB")
    print(f"  Дата: {results['min_currency']['date']}")

    print(f"\n3. СРЕДНИЙ КУРС РУБЛЯ ЗА ВЕСЬ ПЕРИОД:")
    print(f"  Среднее значение: {results['average_rate']:.6f} RUB за 1 единицу валюты")

def main():
    # Получаем данные
    rates_data = get_exchange_rates_for_period(90)
    if not rates_data:
        print("Не удалось получить данные!")
        return

    # Анализируем данные
    results = analyze_currency_data(rates_data)
    if results:
        print_detailed_results(results)


if __name__ == "__main__":
    main()

Сбор данных за последние 90 дней
+ 24/09/2025 (24.09.2025): получено 55 валют (день 1/90)
+ 23/09/2025 (23.09.2025): получено 55 валют (день 2/90)
+ 22/09/2025 (20.09.2025): получено 55 валют (день 3/90)
# 21/09/2025 (20.09.2025) уже обработана, пропуск
# 20/09/2025 уже обработана, предварительный пропуск
+ 19/09/2025 (19.09.2025): получено 55 валют (день 4/90)
+ 18/09/2025 (18.09.2025): получено 55 валют (день 5/90)
+ 17/09/2025 (17.09.2025): получено 55 валют (день 6/90)
+ 16/09/2025 (16.09.2025): получено 55 валют (день 7/90)
+ 15/09/2025 (13.09.2025): получено 55 валют (день 8/90)
# 14/09/2025 (13.09.2025) уже обработана, пропуск
# 13/09/2025 уже обработана, предварительный пропуск
+ 12/09/2025 (12.09.2025): получено 55 валют (день 9/90)
+ 11/09/2025 (11.09.2025): получено 55 валют (день 10/90)
+ 10/09/2025 (10.09.2025): получено 55 валют (день 11/90)
+ 09/09/2025 (09.09.2025): получено 55 валют (день 12/90)
+ 08/09/2025 (06.09.2025): получено 55 валют (день 13/90)
# 07/09/2025 (06